# Hyperparameter Tuning with Optuna

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/USER/teaching-cnn/blob/main/book/exercises/optuna_tuning.ipynb)

Tune learning rate, weight decay, and channels on a toy training loop.

In [ ]:
# Deps
import sys, subprocess
for p in ['optuna', 'torch', 'numpy']:
    try:
        __import__(p)
    except Exception:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', p, '-q'])

In [ ]:
import optuna
import torch, torch.nn as nn, torch.optim as optim
import numpy as np

X = torch.randn(64,1,16,16)
y = torch.randn(64,1,16,16)

class Tiny(nn.Module):
    def __init__(self, ch=8):
        super().__init__()
        self.net = nn.Sequential(nn.Conv2d(1,ch,3,padding=1), nn.ReLU(), nn.Conv2d(ch,1,3,padding=1))
    def forward(self,x): return self.net(x)

criterion = nn.MSELoss()

def objective(trial):
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    wd = trial.suggest_float('weight_decay', 0.0, 1e-3)
    ch = trial.suggest_categorical('channels', [4,8,16,32])
    model = Tiny(ch)
    opt = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    for _ in range(10):
        opt.zero_grad(); loss = criterion(model(X), y); loss.backward(); opt.step()
    return float(loss)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=15)
print('Best:', study.best_params)